In [1]:
!pip install uv

In [2]:
!uv sync

Resolved 76 packages in 1ms
Audited 69 packages in 0.03ms


In [3]:
!source .venv/bin/activate

'source' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


## Load Dataset

In [4]:
from datasets import load_dataset
ds = load_dataset("CADCODER/GenCAD-Code", num_proc=16, split=["train", "test"], cache_dir="/Volumes/BIG-DATA/HUGGINGFACE_CACHE")

d:\第一份工作\mecagent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
ds_train = load_dataset(
    "CADCODER/GenCAD-Code",
    split="train",
    cache_dir="/Volumes/BIG-DATA/HUGGINGFACE_CACHE"
)

ds_test = load_dataset(
    "CADCODER/GenCAD-Code",
    split="test",
    cache_dir="/Volumes/BIG-DATA/HUGGINGFACE_CACHE"
)

In [6]:
print("Length of train set", len(ds_train))
print("Length of test set", len(ds_test))


Length of train set 147289
Length of test set 7355


As we can see, the train set and test set are very large, but due to the limitation of having only a single GPU and local memory constraints, only a small portion of them will be used in subsequent executions.

In [7]:
for i in range(2):
    print(f"\n=== Example {i+1} ===")
    example = ds_train[i]
    for key, value in example.items():
        print(f"{key}: {value}\n")


=== Example 1 ===
image: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x448 at 0x219D8E7DF50>

deepcad_id: 0000/00006371

cadquery: import cadquery as cq
# Generating a workplane for sketch 0
wp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(-0.015625, -0.0078125, 0.0), cq.Vector(1.0, 0.0, 0.0), cq.Vector(0.0, 0.0, 1.0)))
loop0=wp_sketch0.moveTo(0.0, 0.0).threePointArc((0.0007948582418457166, -0.0019189575476279677), (0.0027138157894736844, -0.0027138157894736844)).lineTo(0.021217105263157895, -0.0027138157894736844).threePointArc((0.022787161438489866, -0.00206347722796355), (0.0234375, -0.000493421052631579)).lineTo(0.0234375, 0.018256578947368422).threePointArc((0.02283825686147997, 0.019949990385858287), (0.021217105263157895, 0.020723684210526318)).lineTo(0.0022203947368421052, 0.020723684210526318).threePointArc((0.0005992431385200307, 0.019949990385858287), (0.0, 0.018256578947368422)).lineTo(0.0, 0.0).close()
solid0=wp_sketch0.add(loop0).extrude(0.75)
solid=solid0


token

In [8]:
for i in range(2):
    print(f"\n=== Example {i+1} ===")
    example = ds_test[i]
    for key, value in example.items():
        print(f"{key}: {value}\n")


=== Example 1 ===
image: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x448 at 0x219D8EC4510>

deepcad_id: 0000/00009254

cadquery: import cadquery as cq
# Generating a workplane for sketch 0
wp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(0.0, -0.75, -0.75), cq.Vector(3.749399456654644e-33, 1.0, -6.123233995736766e-17), cq.Vector(1.0, 0.0, 6.123233995736766e-17)))
loop0=wp_sketch0.moveTo(1.5, 0.0).lineTo(1.5, 1.5).lineTo(0.0, 1.5).lineTo(0.0, 0.0).close()
loop1=wp_sketch0.moveTo(0.7578947368421053, 0.5368421052631579).circle(0.14210526315789473)
loop2=wp_sketch0.moveTo(0.7578947368421053, 0.9315789473684211).circle(0.14210526315789473)
solid0=wp_sketch0.add(loop0).add(loop1).add(loop2).extrude(0.03125)
solid=solid0


token_count: 1050

prompt: Generate the CADQuery code needed to create the CAD for the provided image. Just the code, no other words.

hundred_subset: False


=== Example 2 ===
image: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x448 at 0x219D8EC48D0>



## Baseline model

I initially tried CodeLlama but had to abandon it due to GPU and memory limitations. GPT-2, being a much smaller model, is better suited to my computer’s capabilities, although it has limited performance in understanding generative tasks.

My approach is to use CLIP, a model trained to associate images with their matching textual descriptions via contrastive pre-training. During training, CLIP learns to bring the embeddings of an image and its corresponding CadQuery code closer together, while pushing apart embeddings of non-matching pairs.

In theory, by leveraging CLIP to extract image features paired with their corresponding CadQuery code, we can learn a joint representation that aligns the visual CAD images with the procedural CadQuery code generation, improving the model’s understanding and generation accuracy.



In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, CLIPProcessor, CLIPModel
import torch
from torch.utils.data import DataLoader
from PIL import Image
from torch import nn, optim
from metrics.valid_syntax_rate import evaluate_syntax_rate_simple
from metrics.best_iou import get_iou_best
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
# Initiation of CLIP ang gen-AI model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt_model.to(device)
clip_model.to(device)

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# Add the feature of image captured by CLIPmodel in dataset
def preprocess(example):
    image = example["image"]
    if isinstance(image, dict):  
        image = Image.open(image["path"]).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs.to(device))
    image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)  

    tokenized_code = tokenizer(
        example["cadquery"],
        truncation=True,
        max_length=1024,
        padding="max_length",
        return_tensors="pt"
    )

    return {
        "image_features": image_features.squeeze(0),  
        "input_ids": tokenized_code.input_ids.squeeze(0),
        "attention_mask": tokenized_code.attention_mask.squeeze(0),
        "cadquery": example["cadquery"],  
    }


In [35]:
sample = ds_train[0]
result = preprocess(sample)
print("image_features shape:", result["image_features"].shape)
print("input_ids shape:", result["input_ids"].shape)
print("attention_mask shape:", result["attention_mask"].shape)
print("cadquery sample:", result["cadquery"][:100])



image_features shape: torch.Size([512])
input_ids shape: torch.Size([1024])
attention_mask shape: torch.Size([1024])
cadquery sample: import cadquery as cq
# Generating a workplane for sketch 0
wp_sketch0 = cq.Workplane(cq.Plane(cq.Ve


In [ ]:
class CadQueryDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset):
        self.ds = hf_dataset
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        return preprocess(self.ds[idx])

def sample_dataset(dataset, sample_ratio=0.1, seed=42):
    total_size = len(dataset)
    sample_size = int(total_size * sample_ratio)
    random.seed(seed)
    sampled_indices = random.sample(range(total_size), sample_size)
    return Subset(dataset, sampled_indices)


train_dataset = CadQueryDataset(ds_train)
test_dataset = CadQueryDataset(ds_test)

# sampling only 0.2% and batch size 2, as a result of limited memory and gpu
train_sampled_dataset = sample_dataset(train_dataset, sample_ratio=0.002)
test_sampled_dataset = sample_dataset(test_dataset, sample_ratio=0.002)

train_loader = DataLoader(train_sampled_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_sampled_dataset, batch_size=2, shuffle=False)


class CadQueryModel(nn.Module):
    def __init__(self, lm_model, img_feat_dim=512):
        super().__init__()
        self.lm = lm_model
        self.img_proj = nn.Linear(img_feat_dim, lm_model.config.n_embd)
    def forward(self, image_features, input_ids, attention_mask):
        img_embeds = self.img_proj(image_features).unsqueeze(1) 
        inputs_embeds = self.lm.transformer.wte(input_ids)  
        inputs_embeds = torch.cat([img_embeds, inputs_embeds[:, :-1, :]], dim=1)
        outputs = self.lm(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
        return outputs

model = CadQueryModel(gpt_model).to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)


In [ ]:
# Trained only 1 epoch as limited time and gpu
num_epochs = 1
print_every = 10
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for i, batch in enumerate(train_loader):
        image_features = batch["image_features"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(image_features, input_ids, attention_mask)
        logits = outputs.logits

        shift_logits = logits[:, 1:].contiguous()
        shift_labels = input_ids[:, :-1].contiguous()

        loss_fct = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % print_every == 0:
            print(f"Epoch {epoch+1} Batch {i+1}/{len(train_loader)} Loss: {loss.item()}")

Epoch 1/1
Epoch 1 Batch 10/147 Loss: 0.009183360263705254
Epoch 1 Batch 20/147 Loss: 0.0052987574599683285
Epoch 1 Batch 30/147 Loss: 0.003407438052818179
Epoch 1 Batch 40/147 Loss: 0.00040605096728540957
Epoch 1 Batch 50/147 Loss: 0.0015972413821145892
Epoch 1 Batch 60/147 Loss: 0.0006117736338637769
Epoch 1 Batch 70/147 Loss: 0.0023395493626594543
Epoch 1 Batch 80/147 Loss: 0.0037013886030763388
Epoch 1 Batch 90/147 Loss: 0.0005132909282110631
Epoch 1 Batch 100/147 Loss: 0.000938297831453383
Epoch 1 Batch 110/147 Loss: 0.0010637459345161915
Epoch 1 Batch 120/147 Loss: 0.0003731530159711838
Epoch 1 Batch 130/147 Loss: 8.680362952873111e-05
Epoch 1 Batch 140/147 Loss: 0.0001587060687597841


In [ ]:
from tqdm import tqdm

model.eval()
generated_codes = {}
ground_truth_codes = {}

for i, batch in tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating"):
    image_features = batch["image_features"].to(device)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    gt_codes = batch["cadquery"]

    with torch.no_grad():
        img_embeds = model.img_proj(image_features).unsqueeze(1)
        max_len = 256 # Normally should be 1024 but as limited time and only have trained the model 1 time, GPT2 can not really understand the task and only generate nonsense and endless sentence, I tended to stop it early for shorter sentence
        generated = img_embeds

        generated_ids = torch.full(
            (image_features.size(0), 1),
            tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id,
            dtype=torch.long,
            device=device,
        )

        for _ in range(max_len):
            inputs_embeds = torch.cat([img_embeds, model.lm.transformer.wte(generated_ids)[:, :-1, :]], dim=1)
            outputs = model.lm(inputs_embeds=inputs_embeds)
            next_token_logits = outputs.logits[:, -1, :]
            next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            generated_ids = torch.cat([generated_ids, next_token_id], dim=1)
            if all(t.item() == tokenizer.eos_token_id for t in next_token_id):
                break

    for idx in range(image_features.size(0)):
        gen_code = tokenizer.decode(generated_ids[idx].cpu().numpy(), skip_special_tokens=True)
        generated_codes[f"test_{i*image_features.size(0)+idx}"] = gen_code
        ground_truth_codes[f"test_{i*image_features.size(0)+idx}"] = gt_codes[idx]
        

Evaluating: 100%|██████████| 7/7 [02:56<00:00, 25.15s/it]


In [37]:
vsr = evaluate_syntax_rate_simple(generated_codes)
print(f"Valid Syntax Rate on test set: {vsr}")

iou_scores = []
for key in generated_codes.keys():
    gt_code = ground_truth_codes[key]
    gen_code = generated_codes[key]
    iou = get_iou_best(gt_code, gen_code)
    iou_scores.append(iou)

mean_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0.0
print(f"Mean IoU on test set: {mean_iou}")


Valid Syntax Rate on test set: 0.0


ValueError: Error executing script unknown: invalid syntax (<string>, line 1)

As we can see here GPT2 is not that capable to understand well the task in such limited samples and resource.

## Enhancement
The idea is to use a Retrieval-Augmented Generation (RAG) approach: for each training sample, we search for the most similar image features captured by CLIP within the training set. The corresponding CadQuery code of these similar images is then retrieved and appended to the prompt to provide additional context for the generation task in order to improve the performance.

In [46]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from PIL import Image
import random
from tqdm import tqdm


total_size = len(ds_train)
sub_size = max(1, int(total_size * 0.01))
random.seed(42)
sub_indices = random.sample(range(total_size), sub_size)


Train_subset = Subset(ds_train, sub_indices)

image_feature_list = []
cadquery_list = []


for idx in tqdm(sub_indices):
    example = ds_train[idx]
    image = example["image"]
    if isinstance(image, dict):
        image = Image.open(image["path"]).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs.to(device))
    image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
    image_feature_list.append(image_features.squeeze(0).cpu())
    cadquery_list.append(example["cadquery"])

image_feature_bank = torch.stack(image_feature_list)  
cadquery_bank = cadquery_list



class CadQuerySubsetDataset(torch.utils.data.Dataset):
    def __init__(self, full_dataset, indices):
        self.ds = full_dataset
        self.indices = indices
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        example = self.ds[real_idx]

        image = example["image"]
        if isinstance(image, dict):
            image = Image.open(image["path"]).convert("RGB")
        inputs = clip_processor(images=image, return_tensors="pt")
        with torch.no_grad():
            image_features = clip_model.get_image_features(**inputs.to(device))
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

        tokenized_code = tokenizer(
            example["cadquery"],
            truncation=True,
            max_length=1024,
            padding="max_length",
            return_tensors="pt"
        )
        return {
            "image_features": image_features.squeeze(0),
            "input_ids": tokenized_code.input_ids.squeeze(0),
            "attention_mask": tokenized_code.attention_mask.squeeze(0),
            "index": real_idx,
            "cadquery": example["cadquery"],
        }




100%|██████████| 1472/1472 [00:17<00:00, 82.46it/s]


In [ ]:
subsub_size = max(1, int(len(Train_subset) * 0.20))
random.seed(123)
Train_subsub = random.sample(range(len(Train_subset)), sub_size)

train_indices_for_loader = [sub_indices[i] for i in Train_subsub]

train_dataset = CadQuerySubsetDataset(ds_train, train_indices_for_loader)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

model_new = CadQueryModel(gpt_model).to(device)

num_epochs = 1
print_every = 10
topK = 3
loss_fct = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

model_new.train()

index_map = {orig_idx: i for i, orig_idx in enumerate(sub_indices)}

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for i, batch in enumerate(train_loader):
        image_features = batch["image_features"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        indices = batch["index"]  

        image_features_norm = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        bank_norm = image_feature_bank.to(device)
        bank_norm = bank_norm / bank_norm.norm(p=2, dim=-1, keepdim=True)

        sim_matrix = torch.matmul(image_features_norm, bank_norm.T)

        for b, orig_idx in enumerate(indices):
            local_idx = index_map[orig_idx.item()]
            sim_matrix[b, local_idx] = float("-inf")

        topk_sim, topk_idx = torch.topk(sim_matrix, k=topK, dim=-1)

        augmented_prompts = []
        for b in range(len(indices)):
            original_prompt = tokenizer.decode(input_ids[b], skip_special_tokens=True)
            retrieved_cadqueries = [cadquery_bank[j] for j in topk_idx[b].cpu().tolist()]
            retrieved_text = "\n".join(retrieved_cadqueries)
            new_prompt = original_prompt + "\n### Retrieved similar CAD Queries:\n" + retrieved_text
            augmented_prompts.append(new_prompt)

        tokenized_augmented = tokenizer(
            augmented_prompts,
            truncation=True,
            max_length=1024,
            padding="max_length",
            return_tensors="pt"
        )
        input_ids_aug = tokenized_augmented.input_ids.to(device)
        attention_mask_aug = tokenized_augmented.attention_mask.to(device)

        outputs = model(image_features, input_ids_aug, attention_mask_aug)
        logits = outputs.logits

        shift_logits = logits[:, 1:].contiguous()
        shift_labels = input_ids[:, :-1].contiguous()

        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % print_every == 0:
            print(f"Epoch {epoch+1} Batch {i+1}/{len(train_loader)} Loss: {loss.item()}")


Epoch 1/1
Epoch 1 Batch 10/147 Loss: 0.0001393738784827292
Epoch 1 Batch 20/147 Loss: 0.00017399626085534692
Epoch 1 Batch 30/147 Loss: 0.00022302991419564933
Epoch 1 Batch 40/147 Loss: 0.0014955232618376613
Epoch 1 Batch 50/147 Loss: 0.0004334615950938314
Epoch 1 Batch 60/147 Loss: 0.00012110720126656815
Epoch 1 Batch 70/147 Loss: 0.0002595771220512688
Epoch 1 Batch 80/147 Loss: 0.0013687260216102004
Epoch 1 Batch 90/147 Loss: 0.00016199050878640264
Epoch 1 Batch 100/147 Loss: 0.0008383355452679098
Epoch 1 Batch 110/147 Loss: 9.42173574003391e-05
Epoch 1 Batch 120/147 Loss: 8.164272003341466e-05
Epoch 1 Batch 130/147 Loss: 0.0001025463207042776
Epoch 1 Batch 140/147 Loss: 7.57356101530604e-05


In [44]:
from tqdm import tqdm

model_new.eval()
generated_codes = {}
ground_truth_codes = {}

for i, batch in tqdm(enumerate(test_loader), total=len(test_loader), desc="Evaluating"):
    image_features = batch["image_features"].to(device)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    gt_codes = batch["cadquery"]

    with torch.no_grad():
        img_embeds = model_new.img_proj(image_features).unsqueeze(1)
        max_len = 256
        generated = img_embeds

        generated_ids = torch.full(
            (image_features.size(0), 1),
            tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id,
            dtype=torch.long,
            device=device,
        )

        for _ in range(max_len):
            inputs_embeds = torch.cat([img_embeds, model.lm.transformer.wte(generated_ids)[:, :-1, :]], dim=1)
            outputs = model.lm(inputs_embeds=inputs_embeds)
            next_token_logits = outputs.logits[:, -1, :]
            next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            generated_ids = torch.cat([generated_ids, next_token_id], dim=1)
            if all(t.item() == tokenizer.eos_token_id for t in next_token_id):
                break

    for idx in range(image_features.size(0)):
        gen_code = tokenizer.decode(generated_ids[idx].cpu().numpy(), skip_special_tokens=True)
        generated_codes[f"test_{i*image_features.size(0)+idx}"] = gen_code
        ground_truth_codes[f"test_{i*image_features.size(0)+idx}"] = gt_codes[idx]
        print("The generated CADquery :",generated_codes)
        print("The ground truths: ", ground_truth_codes)

Evaluating:  14%|█▍        | 1/7 [00:25<02:33, 25.54s/it]

The generated CADquery : {'test_0': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................'}
The ground truths:  {'test_0': 'import cadquery as cq\n# Generating a workplane for sketch 0\nwp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(-0.4453125, 0.0, -0.71875), cq.Vector(1.0, 6.123233995736766e-17, -6.123233995736766e-17), cq.Vector(6.123233995736766e-17, -1.0, 6.123233995736766e-17)))\nloop0=wp_sketch0.moveTo(0.8927631578947368, 0.0).lineTo(0.8927631578947368, 1.4375).lineTo(0.0, 1.4375).lineTo(0.0, 0.0).close()\nsolid0=wp_sketch0.add(loop0).extrude(0.0234375)\nsolid=solid0\n'}
The generated CADquery : {'test_0': '.The .. .. .. .. .. ....................................................................................................................................................

Evaluating:  29%|██▊       | 2/7 [00:50<02:06, 25.29s/it]

The generated CADquery : {'test_0': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_1': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_2': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................'}
The ground truths:  {'test_0': 'import cadquery as cq\n# Generating a workplane for sketch 0\nwp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(-

Evaluating:  43%|████▎     | 3/7 [01:15<01:40, 25.05s/it]

...

The generated CADquery : {'test_0': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_1': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_2': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_3': '.The .. .. .. .. .. .............................................................................................................

Evaluating:  86%|████████▌ | 6/7 [02:39<00:27, 27.46s/it]

The generated CADquery : {'test_0': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_1': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_2': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_3': '.The .. .. .. .. .. .............................................................................................................

Evaluating: 100%|██████████| 7/7 [03:04<00:00, 26.37s/it]

The generated CADquery : {'test_0': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_1': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_2': '.The .. .. .. .. .. ....................................................................................................................................................................................................................................................', 'test_3': '.The .. .. .. .. .. .............................................................................................................

In [42]:
vsr = evaluate_syntax_rate_simple(generated_codes)
print(f"Valid Syntax Rate on test set: {vsr}")

iou_scores = []
for key in generated_codes.keys():
    gt_code = ground_truth_codes[key]
    gen_code = generated_codes[key]
    iou = get_iou_best(gt_code, gen_code)
    iou_scores.append(iou)

mean_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0.0
print(f"Mean IoU on test set: {mean_iou}")

Valid Syntax Rate on test set: 0.0


ValueError: Error executing script unknown: invalid syntax (<string>, line 1)